## Fake News Prediction

### Workflow

#### 1. collect data
#### 2. Data preprcessing
#### 3. Test test split
#### 4. Logstic regression model
#### 5. Evaluate the data
#### 6. Prediction

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
data = pd.read_csv("train.csv")

In [8]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
## doesnot add much value in prediction

print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Preprocessing

In [11]:
data.shape

(20800, 5)

In [12]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [13]:
### Replacing the null values with empty string

data = data.fillna('')

In [14]:
## Mergigng titile and author name

data['content'] = data['author'] + ' '+data['title']

In [15]:
data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [16]:
### We will use content as features and label as label for prediction

X = data.drop(columns = 'label', axis = 1)
Y = data['label']

In [17]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [18]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

### Steming 

#### Steming is a process of reducing a word to its root word

In [19]:
### Example

# actor, actress, acting ----> act

port_stem = PorterStemmer()

In [26]:
### for searching the paragragh

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)    ## remove everyhting apart from alphabet
    stemmed_cotent = stemmed_content.lower()
    stemmed_cotent = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
data['content'] = data['content'].apply(stemming)

In [24]:
## Seperating the data and label

X = data['content'].values
Y = data['label'].values

In [25]:
print(X)

['d r r e l l   l u c u   h u e   d e   a e     w e   d n     e v e n   s e e   c e     l e e r   u n l   j n   c h f f e z   t w e e e   i'
 'd n e l   j     f l n n   f l y n n     h l l r   c l n n     b g   w n   n   c p u       b r e b r'
 'c n r u n e w   c   w h   h e   t r u h   m g h   g e   y u   f r e' ...
 'm c h e l   j     e   l   m e r c e   n   r c h e l   a b r   m c     i   s     r e c e v e   t k e v e r   a p p r c h   b   h u n     b       t h e   n e w   y r k   t e'
 'a l e x   a n r   n a t o     r u   t   h l   p r l l e l   e x e r c e   i n   b l k n'
 'd v   s w n n   w h   k e e p   h e   f         a l v e']


In [ ]:
## converting the textual data to numerical data

vectorizer = TfidVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state=2)

### Training the Model

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(xtrain, Ytrain)

### Evaluation

#### Accuracy

In [ ]:
## Accuracy score on the testing data


xtrain_prediction = model.predict(xtrain)
training_data_accuracy = accuracy_score(xtrain_prediction, ytrain)

In [ ]:
xtest_prediction = model.predict(xtest)
test_data_accuracy = accuracy_score(xtest_prediction, ytest)

## prediction on new data

In [ ]:
Xnew = xtest[0]

prediction = model.predict(Xnew)
print(prediction)

if prediction[0]==0:
    print("The news is real")
    
else:
    print('The news is Fake')